In [1]:
# Script créé par Julien Caugant, bibliothécaire-assistant à Aix-Marseille Université, sous licence GNU-GPL
import PyPDF2, requests as rq, json, pandas as pd
import urllib.request
import os
import re, time
keyword = ["GitHub","GitLab","Github","Gitlab"]
k_found = []
hal_id = []
submit_by = []

In [3]:
url = "https://api.archives-ouvertes.fr/search/?q=*:*&wt=json&rows=500&fq=submitType_s:file&fq=structId_i:198056&fq=submittedDateY_i:2024&fl=uri_s,contributorFullName_s"

In [5]:
req = rq.get(url)
req = req.json()

In [7]:
for i in range(0, len(req['response']['docs'])):
    try :
        pdf = req['response']['docs'][i]['uri_s']+"/document"
        sub = req['response']['docs'][i]['contributorFullName_s']
        urllib.request.urlretrieve(pdf, "doc.pdf")
        read_pdf = open('doc.pdf', 'rb')
        try :
            pdfReader = PyPDF2.PdfReader(read_pdf)
            for j in range(0,len(pdfReader.pages)):
                pageObj = pdfReader.pages[j]
                text=(pageObj.extract_text())
                text=text.split(",")
                for word in text :
                    for k in keyword :
                        if k in word :
                            print(f"Le terme {k} a été trouvé dans le document {pdf}")
                            k_found.append(k)
                            hal_id.append(pdf)
                            submit_by.append(sub)
        except :
            continue
    except :
        continue
columns = ['Terme trouvé','Identifiant HAL','Déposant']
df = pd.DataFrame(columns=columns)
df['Terme trouvé'], df['Identifiant HAL'], df['Déposant'] = k_found, hal_id, submit_by
df.drop_duplicates(subset ="Identifiant HAL", keep = 'first', inplace=True)

        # text=text.split(",")
        # for word in text :
            # print(word)
            # if word == keyword :
                # print(f"Le terme {keyword} a été trouvé dans le document {pdf}")
    # read_pdf.close()
                
            
    

Le terme GitHub a été trouvé dans le document https://hal.science/hal-04652803v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04652803v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04230475v1/document
Le terme GitHub a été trouvé dans le document https://amu.hal.science/hal-04469832v1/document
Le terme GitHub a été trouvé dans le document https://amu.hal.science/hal-04469832v1/document
Le terme GitHub a été trouvé dans le document https://inrap.hal.science/hal-04113420v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04424326v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-03833331v1/document


Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in dictionary at byte 0x7d for key /Subtype
Multiple definitions in d

Le terme GitHub a été trouvé dans le document https://hal.science/hal-04526933v1/document
Le terme GitHub a été trouvé dans le document https://insu.hal.science/insu-04479091v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme Gitlab a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04415353v1/document
Le terme GitHub a été trouvé dans le document https://hal.science/hal-04488053v1/document
Le t

In [9]:
df

,Terme trouvé,Identifiant HAL,Déposant
0,GitHub,https://hal.science/hal-04652803v1/document,Christophe Leterrier
2,GitHub,https://hal.science/hal-04230475v1/document,Alaric Tabariès
3,GitHub,https://amu.hal.science/hal-04469832v1/document,Nathalie BONNARDEL
5,GitHub,https://inrap.hal.science/hal-04113420v1/document,Luc Sanson
6,GitHub,https://hal.science/hal-04424326v1/document,LEST UMR 7317
7,GitHub,https://hal.science/hal-03833331v1/document,Nathalie POTHIER
8,GitHub,https://hal.science/hal-04526933v1/document,Julien Caugant
9,GitHub,https://insu.hal.science/insu-04479091v1/document,Nathalie POTHIER
10,GitHub,https://hal.science/hal-04415353v1/document,Joseph MARIANI
18,GitHub,https://hal.science/hal-04488053v1/document,Julie BEUGIN


In [11]:
df.to_csv("result.csv")

In [ ]:
pdf